In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
sess = tf.Session()
off_rtg = tf.placeholder(dtype=tf.float32)

In [3]:
W_off_rtg = tf.Variable([np.random.random_sample(), np.random.random_sample()])
b_off_rtg = tf.Variable([-1*np.random.random_sample(), -1*np.random.random_sample()])

In [4]:
model = tf.nn.softmax(off_rtg*W_off_rtg + b_off_rtg)
y = tf.placeholder(tf.float32)

In [5]:
init = tf.global_variables_initializer()
sess.run(init)

In [6]:
mse = tf.reduce_mean(tf.squared_difference(model, y))
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(mse)

In [21]:
# training data
#off_train = [[92.7,89.5],[104.1,90.4],[117.0,127.5],[117.0,127.5]]
#y_train = [[1,0],[1,0],[0,1],[0,1]]

off_train = []
y_train = []

test_data = pd.read_csv("data/test.csv", delimiter=',', header=None).values.tolist()

for data in test_data:
    off_train.append([data[0],data[1]])
    y_train.append([data[2],data[3]])

off_train = off_train[:400]
y_train = y_train[:400]

In [22]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

sess.run(train, {off_rtg: off_train, y: y_train})

In [23]:
off_ratings = pd.Series.from_csv("data/current_offrtg.csv", header=None).to_dict()
games_tonight = pd.read_csv("data/games_20012018.csv", delimiter=',', header=None).values.tolist()
print(games_tonight)

[['Jan', 20, 2018, '7:00 pm', 'Miami Heat', 'Charlotte Hornets'], ['Jan', 20, 2018, '7:00 pm', 'Memphis Grizzlies', 'New Orleans Pelicans'], ['Jan', 20, 2018, '7:30 pm', 'Milwaukee Bucks', 'Philadelphia 76ers'], ['Jan', 20, 2018, '8:30 pm', 'Golden State Warriors', 'Houston Rockets'], ['Jan', 20, 2018, '9:00 pm', 'Toronto Raptors', 'Minnesota Timberwolves'], ['Jan', 20, 2018, '9:00 pm', 'Los Angeles Clippers', 'Utah Jazz'], ['Jan', 20, 2018, '10:00 pm', 'Dallas Mavericks', 'Portland Trail Blazers']]


In [24]:
def make_prediction(game):
    away_team = [game[4],off_ratings[game[4]]]
    home_team = [game[5],off_ratings[game[5]]]
    results = sess.run(model, feed_dict={off_rtg: [away_team[1],home_team[1]]})
    if results[0] >= results[1]:
        result = "away"
        confidence = results[0]
    else:
        result = "home"
        confidence = results[1]
    
    return [result, confidence]

In [25]:
for game in games_tonight:
    result, confidence = make_prediction(game)
    game.append(result)
    game.append(confidence)

In [26]:
print(games_tonight)

[['Jan', 20, 2018, '7:00 pm', 'Miami Heat', 'Charlotte Hornets', 'home', 0.99999678], ['Jan', 20, 2018, '7:00 pm', 'Memphis Grizzlies', 'New Orleans Pelicans', 'home', 1.0], ['Jan', 20, 2018, '7:30 pm', 'Milwaukee Bucks', 'Philadelphia 76ers', 'home', 0.99993765], ['Jan', 20, 2018, '8:30 pm', 'Golden State Warriors', 'Houston Rockets', 'home', 0.99999571], ['Jan', 20, 2018, '9:00 pm', 'Toronto Raptors', 'Minnesota Timberwolves', 'home', 0.99999857], ['Jan', 20, 2018, '9:00 pm', 'Los Angeles Clippers', 'Utah Jazz', 'home', 0.99995875], ['Jan', 20, 2018, '10:00 pm', 'Dallas Mavericks', 'Portland Trail Blazers', 'home', 0.99999213]]


In [27]:
df = pd.DataFrame(games_tonight, columns=["Month","Day","Year","Start (ET)","Visitor","Home","Outcome","Confidence"])
df.to_csv('results.csv', index=False)